In [4]:
!pip install opencv-python
!pip install torch
!pip install yolo5

  Using cached yolo5-0.0.1-py36.py37.py38-none-any.whl.metadata (6.0 kB)
  Using cached yolov5-7.0.13-py37.py38.py39.py310-none-any.whl.metadata (10 kB)
  Using cached fire-0.6.0-py2.py3-none-any.whl
  Using cached boto3-1.34.71-py3-none-any.whl.metadata (6.6 kB)
  Using cached sahi-0.11.15-py3-none-any.whl.metadata (15 kB)
  Using cached huggingface_hub-0.22.1-py3-none-any.whl.metadata (12 kB)
  Using cached roboflow-1.1.26-py3-none-any.whl.metadata (9.3 kB)
  Using cached botocore-1.34.71-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.10.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached certifi-2023.7.22-py3-none-any.whl.metadata (2.2 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl.metadata (722 bytes)
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached opencv_python_headless-4.8.0.74-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl.metadata (18 kB)
Using cached yolo5-0.0.1-py36.p

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\arvi3\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [3]:
import torch
import cv2
from PIL import Image
import numpy as np
from torchvision.transforms import functional as F
from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression, scale_coords
from yolov5.utils.torch_utils import select_device

def load_yolov5_model(weights_path):
    device = select_device('')
    model = attempt_load(weights_path, map_location=device)
    return model

def preprocess_image(img):
    img = np.array(img)
    img = img[:, :, ::-1]  # BGR to RGB
    img = Image.fromarray(img)
    img = F.resize(img, (640, 640))
    img_tensor = F.to_tensor(img).float()
    img_tensor /= 255.0
    img_tensor = img_tensor.unsqueeze(0)
    return img_tensor

def predict(model, img_tensor, conf_threshold=0.25):
    with torch.no_grad():
        detections = model(img_tensor)[0]
        detections = non_max_suppression(detections, conf_threshold, 0.45)

    return detections[0] if detections[0] is not None else []

def draw_bbox(image, bbox, color=(0, 255, 0), thickness=2):
    for x1, y1, x2, y2, conf, cls_conf, cls_pred in bbox:
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
        text = f'Butterfly: {conf:.2f}'
        cv2.putText(image, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness)

# Load YOLOv5 model
weights_path = 'path_to_your_yolov5_weights.pt'
model = load_yolov5_model(weights_path)

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess image
    img_tensor = preprocess_image(frame)

    # Perform inference
    detections = predict(model, img_tensor)

    # Draw bounding boxes
    if len(detections) > 0:
        draw_bbox(frame, detections)

    # Display output
    cv2.imshow('YOLOv5 Butterfly Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'yolov5'